# COURSERA ASSIGNMENT WEEK 3

In [1]:
#Import all the necessary libraries
import numpy as np 
import pandas as pd
import json 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 
from bs4 import BeautifulSoup

## Q1: Scraping data from the Wikipedia page

First, we use the BeautifulSoup library to read the page html. Looking at the html, we find that the table is under the class 'wikitable sortable'. Using the find method, we pass this to a variable called table.

In [2]:
wiki_link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#get the html and convert to parser
web_html = requests.get(wiki_link).text
soup = BeautifulSoup(web_html, 'html.parser')

#the table is under the class wikitable sortable
table = soup.find('table', {'class':'wikitable sortable'})

In the table, the rows are designated by 'tr' and each cell in the row is designated by 'td'. Iterating over each row, we pass the first element of the cell to the pin code variable, the second to the borough variable and the third to the neighborhood variable. We then convert this array into a pandas dataframe, filter out results where the borough is not 'Not Assigned', and then replace the '/' with a comma.

In [3]:
#create blank arrays to hold the data
postal_code = []
borough = []
neighborhood = []

rows = table.find_all('tr') #find rows with the TR tag

for r in rows:
    cell = r.find_all('td') #find cells with the TD tag
    if (len(cell) > 1):
        pcode = cell[0] #first element is the postal code
        brgh = cell[1] #second element is the borough name
        nbhd = cell[2] #third element is the neighborhood name
        
        if (brgh != 'Not assigned')&(nbhd == ''): #Assigned borough but no neighborhood
            nbhd = brgh #assign the neighborhood as the borough
        postal_code.append(pcode.text.strip())
        borough.append(brgh.text.strip())
        neighborhood.append(nbhd.text.strip())

#create a dataframe with the arrays
df_pbn = pd.DataFrame({'PostalCode':postal_code, 'Borough':borough, 'Neighborhood':neighborhood})

#check which Borough value is not Not assigned and pass to new dataframe
filter = df_pbn['Borough'] != 'Not assigned'
df_pbn = df_pbn[filter]

#replace the slash with a comma
for index, rowdata in df_pbn.iterrows():
    df_pbn.at[index, 'Neighborhood'] = df_pbn.at[index, 'Neighborhood'].replace(' /', ',')
    
df_pbn

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road , Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,Business reply mail Processing CentrE
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


## Q2: Add latitude and longitude data

We first read the CSV from the link provided. The resulting dataframe has Postal Codr, Latitude and Longitude data. We can construct the final table by doing a left join of the previous dataframe with this one, on the postal code keys,

In [4]:
link = 'http://cocl.us/Geospatial_data'
df_pll = pd.read_csv(link)
df_pll.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [5]:
#join the two dataframes on the postalcode keys
df_pbnll = df_pbn.merge(df_pll, how = 'left', left_on = 'PostalCode', right_on = 'Postal Code')
df_pbnll.drop('Postal Code', axis = 1, inplace = True)
df_pbnll.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


# Q3: Clustering the neighborhoods

Using the client ID and client secret values, we write a function that iterates over all (latitude, longitude) values in the resultant array from the previous question. For each neighborhood, venues and their latitude and longitude values are read from the JSON and stored to the variable venue_list

In [6]:
CLIENT_ID = '2V5WZC1SXU1MWC221NSQHMHGQUER3OJUEHT4PIO4YPLRJUCS' 
CLIENT_SECRET = 'CTZJTPATBDXTP5ZAWB22I2X2LH4FUESVFNG5FHHEOGFZFYIR' 
VERSION = '20180605' # Foursquare API version
LIMIT = 100
radius = 500

names = df_pbnll['Neighborhood']
latitudes = df_pbnll['Latitude']
longitudes = df_pbnll['Longitude']

venue_list = []

for name, lat, long in zip(names, latitudes, longitudes):
    
    #create the api request
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        long, 
        radius, 
        LIMIT)
    
    #make the get request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for v in results:
        venue_list.append([name, 
                            lat, 
                            long, 
                            v['venue']['name'], 
                            v['venue']['location']['lat'], 
                            v['venue']['location']['lng'],
                            v['venue']['categories'][0]['name']])

The venue_list variable is converted to a pandas dataframe. The 'Venue Category' column is converted from categorical to dummy variables. Non essential data is removed from this dataframe, and then the data is grouped by neighborhoods, with the average value of each dummy/encoded variable

In [46]:
#create a dataframe
df_venues = pd.DataFrame(venue_list)
columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue Name', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
df_venues.columns = columns

#convert categorical data to dummy data using the Venue Category column
cat_columns = ['Venue Category']
df_venues_cat = pd.get_dummies(df_venues, columns = cat_columns)

#remove non essential data columns
columns_rem = ['Venue Name', 'Venue Latitude', 'Venue Longitude']
df_venues_cat.drop(columns_rem, axis = 1, inplace = True)

#group data by neighborhood
df_venues_grouped = df_venues_cat.groupby(['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude']).mean().reset_index()
df_venues_grouped.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Category_Accessories Store,Venue Category_Airport,Venue Category_Airport Food Court,Venue Category_Airport Gate,Venue Category_Airport Lounge,Venue Category_Airport Service,Venue Category_Airport Terminal,...,Venue Category_Vegetarian / Vegan Restaurant,Venue Category_Video Game Store,Venue Category_Video Store,Venue Category_Vietnamese Restaurant,Venue Category_Warehouse Store,Venue Category_Wine Bar,Venue Category_Wine Shop,Venue Category_Wings Joint,Venue Category_Women's Store,Venue Category_Yoga Studio
0,Agincourt,43.794200,-79.262029,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",43.602414,-79.543484,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.052632,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,43.786947,-79.385975,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",43.733283,-79.419750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


We now run the k means clustering method on this grouped dataset. A total of 4 clusters are declared. The labels obtained from k-means are added to the original dataset as the column 'Cluster'.

In [60]:
#creating clusters
x = df_venues_grouped.drop(['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude'], axis = 1)
num_clusters = 4
k_means = KMeans(n_clusters = num_clusters, init = 'k-means++', n_init = 300)
k_means.fit(x)
labels = k_means.labels_
df_venues_grouped['Cluster'] = labels+1
df_venues_grouped['Cluster'].value_counts()

1    84
2     9
4     3
3     2
Name: Cluster, dtype: int64

The results are put into a dataframe for easier plotting. The four clusters are mapped as per the following rules
<br>Cluster 1 - Red (84 items)
<br>Cluster 2 - Blue (9 items)
<br>Cluster 3 - Green (2 items)
<br>Cluster 4 - Black (3 items)

In [62]:
results = pd.DataFrame()
results['N'] = df_venues_grouped['Neighborhood']
results['NLa'] = df_venues_grouped['Neighborhood Latitude']
results['NLo'] = df_venues_grouped['Neighborhood Longitude']
results['C'] = df_venues_grouped['Cluster']

colors = ['red', 'blue', 'green', 'black']

latitude = 43.6532
longitude = -79.3832
map_toronto = folium.Map(location=[latitude, longitude], zoom_start = 10)

for nbh, lat, long, cls in zip(results['N'], results['NLa'], results['NLo'], results['C']):
    label = str(nbh) + ' Cluster ' + str(cls)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, long],
        radius = 5,
        popup = label,
        color = colors[cls-1],
        fill = True,
        fill_color = colors[cls-1],
        fill_opacity = 0.6,
        parse_html = False).add_to(map_toronto)

map_toronto

### Note: Since the folium maps do not render properly on github, the static image has been uploaded to the repository under the name Toronto_Output

We now need to see what are the top 5 venues in each cluster. To do that we create a new dataframe called clusterdata and follow the following steps
<br>1. Filter out the neighborhoods corresponding to the cluster
<br>2. Take a sum of all columns to get the sum of average values of each venue categorical variable
<br>3. Sort these in descending order ad create a dataframe
<br>4. Select the top 5 rows of this dataframe and pass them to a new column in the clusterdata dataframe
<br>5. Repeat steps 1-4 for all clusters

In [58]:
drop_columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude']
x = df_venues_grouped.drop(drop_columns, axis = 1)
clusterdata = pd.DataFrame()

for i in range(0, num_clusters):
    list_i = x[x['Cluster'] == i].drop('Cluster', axis = 1).sum().sort_values(ascending = False).to_frame().reset_index().head()['index']
    cname = 'Cluster ' + str(i)
    clusterdata[cname] = list_i

for index, rowdata in clusterdata.iterrows():
    for column in clusterdata.columns:
        clusterdata.at[index, column] = clusterdata.at[index, column].replace('Venue Category_', '')

clusterdata

,Cluster 0,Cluster 1,Cluster 2
0,Yoga Studio,Coffee Shop,Park
1,Dim Sum Restaurant,Pizza Place,Coffee Shop
2,Fast Food Restaurant,Café,Playground
3,Farmers Market,Park,Convenience Store
4,Falafel Restaurant,Sandwich Place,Airport
